<a href="https://colab.research.google.com/github/qingtaozhou/data_incubator/blob/master/Feature_Engineering(Jan_2020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from plotly import tools, subplots
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
from bokeh.models import Panel, Tabs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import lightgbm as lgb
import plotly.figure_factory as ff
import gc
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%%time
print('Reading train.csv file....')
# train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
train = pd.read_csv('drive/My Drive/data-science-bowl-2019/train.csv')
print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

print('Reading test.csv file....')
# test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
test = pd.read_csv('drive/My Drive/data-science-bowl-2019/test.csv')
print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))

print('Reading train_labels.csv file....')
# train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
train_labels = pd.read_csv('drive/My Drive/data-science-bowl-2019/train_labels.csv')
print('Train_labels.csv file have {} rows and {} columns'.format(train_labels.shape[0], train_labels.shape[1]))

print('Reading specs.csv file....')
# specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
specs = pd.read_csv('drive/My Drive/data-science-bowl-2019/specs.csv')
print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))

print('Reading sample_submission.csv file....')
# sample_submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
sample_submission = pd.read_csv('drive/My Drive/data-science-bowl-2019/sample_submission.csv')
print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))


Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns
Reading specs.csv file....
Specs.csv file have 386 rows and 3 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 1000 rows and 2 columns
CPU times: user 56.7 s, sys: 6.91 s, total: 1min 3s
Wall time: 1min 56s


In [0]:
train['train_flag']=1
test['train_flag']=0

In [0]:
full_data=pd.concat([train,test])

In [0]:
def remove_redundancy(df)->pd.DataFrame:
    df_assessment=df[df['type']=='Assessment']
    column_list=list(df.columns)
    df_assessment=df_assessment[((df_assessment['title']!='Bird Measurer (Assessment)')&(df_assessment['event_code']==4100))\
                                          | ((df_assessment['title']=='Bird Measurer (Assessment)')&(df_assessment['event_code']==4110))]
    correct=df_assessment[df_assessment['event_data'].apply(str).str.contains('"correct":true')]
    first_correct_timestamp=correct.groupby(['game_session','installation_id','title'])['timestamp'].min()
    first_correct_timestamp=first_correct_timestamp.reset_index()
    df=df.merge(first_correct_timestamp,how='left',left_on=['game_session','installation_id','title'],right_on=['game_session','installation_id','title'])
    df_correct_filter=df[df.timestamp_x<=df.timestamp_y]
    df_null=df[df.timestamp_y.isnull()]
    new_df=pd.concat([df_correct_filter,df_null])
    new_df=new_df.drop(['timestamp_y'],axis=1)
    new_df.columns=column_list
    return new_df

In [0]:
sample_data=full_data.sample(17000)
new_sample=remove_redundancy(sample_data)
new_sample.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,train_flag
557,392e14df,d9ab7866b007e19d,2019-10-10T12:04:14.641Z,"{""buckets"":[0,0,0],""correct"":true,""buckets_pla...",a07fb570,13,4100,15879,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,1
595,25fa8af4,e14b23c3a66e1886,2019-09-10T22:33:15.664Z,"{""correct"":true,""stumps"":[2,3,5],""event_count""...",a6769468,31,4100,25488,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,1
2146,392e14df,81c0e06ff9cdbcc4,2019-08-25T16:05:12.404Z,"{""buckets"":[0,0,0],""correct"":true,""buckets_pla...",2ff7b20e,34,4100,35096,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,1
2207,93b353f2,eaabf00784d7afb3,2019-09-22T21:55:51.298Z,"{""correct"":true,""pillars"":[1,2,3],""event_count...",0d09ed83,74,4100,80727,Chest Sorter (Assessment),Assessment,CRYSTALCAVES,1
2352,25fa8af4,51c43a9cc89f27ed,2019-08-21T14:00:42.305Z,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",be9e7422,26,4100,18779,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,1


In [0]:
def get_data(user_sample)->pd.DataFrame:
    user_sample['timestamp'] = pd.to_datetime(user_sample['timestamp'], format="%Y-%m-%d %H:%M:%S")
    user_sample['year']=user_sample['timestamp'].apply(lambda x:x.year) 
    user_sample['month']=user_sample['timestamp'].apply(lambda x:x.month)
    user_sample['day']=user_sample['timestamp'].apply(lambda x:x.day)
    user_sample['dayofweek']=user_sample['timestamp'].apply(lambda x:x.dayofweek)
    user_sample['dayofyear']=user_sample['timestamp'].apply(lambda x:x.dayofyear)
    user_sample['weekofyear']=user_sample['timestamp'].apply(lambda x:x.weekofyear)
    user_sample['hour']=user_sample['timestamp'].apply(lambda x:x.hour)
    user_sample['epoch']=user_sample['timestamp'].apply(lambda x:x.value//10**9)
    user_sample.drop(['timestamp'],axis=1,inplace=True)
    

    event_id=user_sample["event_id"].value_counts()
    game_session=user_sample["game_session"].value_counts()
    #installation_id=user_sample["installation_id"].value_counts()
    #event_count=user_sample["event_count"].value_counts()
    event_code=user_sample["event_code"].value_counts()
    title=user_sample["title"].value_counts()
    type_count=user_sample["type"].value_counts()
    world=user_sample["world"].value_counts()
    dayofyear=user_sample["dayofyear"].value_counts()
    weekofyear=user_sample["weekofyear"].value_counts()

    user_sample["event_id_count"]=user_sample["event_id"].apply(lambda x:event_id[x])
    user_sample["game_session_count"]=user_sample["game_session"].apply(lambda x:game_session[x])  
    # user_sample["installation_id_count"]=user_sample["installation_id"].apply(lambda x:installation_id[x])
    #user_sample["event_count2"]=user_sample["event_count"].apply(lambda x:event_count[x])
    user_sample["title_count"]=user_sample["title"].apply(lambda x:title[x])
    user_sample["type_count"]=user_sample["type"].apply(lambda x:type_count[x])
    user_sample["world_count"]=user_sample["world"].apply(lambda x:world[x])
    user_sample["dayofyear_count"]=user_sample["dayofyear"].\
                                         apply(lambda x:dayofyear[x])
    user_sample["weekofyear_count"]=user_sample["weekofyear"].\
                                         apply(lambda x:weekofyear[x])
    
    user_sample['min_game_time']= user_sample['game_time'].min()
    user_sample['max_game_time']= user_sample['game_time'].max()
    user_sample['mean_game_time']= user_sample['game_time'].mean()
    user_sample['median_game_time']= user_sample['game_time'].median()
    user_sample['min_event_count']= user_sample['event_count'].min()
    user_sample['max_event_count']= user_sample['event_count'].max()
    user_sample['mean_event_count']= user_sample['event_count'].mean()
    user_sample['median_event_count']= user_sample['event_count'].median()


    
    user_assessment=user_sample[((user_sample['title']!='Bird Measurer (Assessment)')&(user_sample['event_code']==4100))\
                                          | ((user_sample['title']=='Bird Measurer (Assessment)')&(user_sample['event_code']==4110))]
    user_assessment['correct']=0
    user_assessment['incorrect']=0
    user_assessment.loc[user_sample['event_data'].apply(str).str.contains('"correct":true'),'correct']=1
    user_assessment.loc[user_sample['event_data'].apply(str).str.contains('"correct":false'),'incorrect']=1
    num_correct=user_assessment.groupby(['game_session','title'])['correct'].sum()
    num_correct=pd.DataFrame(num_correct)
    num_correct.columns=['num_correct']
    num_incorrect=user_assessment.groupby(['game_session','title'])['incorrect'].sum()
    num_incorrect=pd.DataFrame(num_incorrect)
    num_incorrect.columns=['num_incorrect']    
    new_assessment=pd.concat([num_correct,num_incorrect],join='outer',axis=1)
    new_assessment = new_assessment.reset_index()
    new_assessment['accuracy']=new_assessment['num_correct']/(new_assessment['num_correct']+new_assessment['num_incorrect'])
    user_sample=user_sample.merge(new_assessment,on=['game_session','title'],how='outer')
    user_sample=user_sample.fillna(0)
    user_sample['accuracy_group']=user_sample['accuracy'].apply(lambda x: 0 if x==0 else (3 if x==1 else (2 if x==0.5 else 1)) )

    user_sample.drop(['event_id','game_session','event_data','event_code','title','type','world'],axis=1,inplace=True)
    
    return user_sample

In [0]:
from tqdm import tqdm
total_data=pd.DataFrame()
for i, (ins_id, user_sample) in tqdm(enumerate(new_sample.groupby('installation_id', sort=False))):#, total=17000):
      total_data=pd.concat([total_data,get_data(user_sample)])
total_data.head()

5964it [06:37, 14.84it/s]


,installation_id,event_count,game_time,train_flag,year,month,day,dayofweek,dayofyear,weekofyear,hour,epoch,event_id_count,game_session_count,title_count,type_count,world_count,dayofyear_count,weekofyear_count,min_game_time,max_game_time,mean_game_time,median_game_time,min_event_count,max_event_count,mean_event_count,median_event_count,num_correct,num_incorrect,accuracy,accuracy_group
0,a07fb570,13,15879,1,2019,10,10,3,283,41,12,1570709054,1,1,1,2,1,4,4,15076,108846,56024.0,50087.0,13,76,39.75,35.0,1.0,0.0,1.0,3
1,a07fb570,21,15076,1,2019,10,10,3,283,41,12,1570709557,1,1,1,2,3,4,4,15076,108846,56024.0,50087.0,13,76,39.75,35.0,0.0,0.0,0.0,0
2,a07fb570,76,84295,1,2019,10,10,3,283,41,12,1570709922,1,1,1,2,3,4,4,15076,108846,56024.0,50087.0,13,76,39.75,35.0,0.0,0.0,0.0,0
3,a07fb570,49,108846,1,2019,10,10,3,283,41,12,1570709704,1,1,1,2,3,4,4,15076,108846,56024.0,50087.0,13,76,39.75,35.0,0.0,0.0,0.0,0
0,a6769468,31,25488,1,2019,9,10,1,253,37,22,1568154795,1,1,1,3,2,1,1,0,77089,25817.2,16131.0,1,54,24.00,18.0,1.0,0.0,1.0,3


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection, preprocessing
from sklearn.model_selection import GridSearchCV as GSCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as  RFC
from sklearn import model_selection, preprocessing
from xgboost import XGBClassifier as xgb
from sklearn.ensemble import AdaBoostClassifier as ABC

In [0]:
#all_features = [x for x in total_data.columns if x not in (['accuracy_group'] and ['installation_id'] and ['accuray'])]
all_features = [x for x in total_data.columns if x not in (['accuracy_group'] and ['accuray'])]
#print(all_features)
X, y = total_data[all_features],total_data['accuracy_group']
#X.drop(['installation_id', 'accuracy'], axis=1,inplace=True)
#X = StandardScaler().fit_transform(X)
#X.info()
print(X.shape)
m = X['accuracy_group']
print((X['accuracy_group']==0).sum())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.4)
results=[]
results = X_test['installation_id']
X_train.drop(['installation_id', 'accuracy'], axis=1,inplace=True)
X_test.drop(['installation_id', 'accuracy'], axis=1,inplace=True)
#print(results.shape[0])
#print(results)
#results.to_csv('output.csv')

(16999, 31)
16919


In [0]:
def train_the_model (model):
  #--------selecting the model that is interested........................
  if model == "RandomForest":
    model_run=RFC()
    params=[]
  elif model =="Adaboost":
    model_run=ABC()
    paramGrid = {'learning_rate': [0.001,0.01,0.1],'n_estimators': [50,100,200]}
    #gridsearch=GSCV(model_run,paramGrid,verbose=1,cv=KFold(n_splits=10,random_state=rstate).get_n_splits([X_train,y_train]))
    #gridsearch.fit(X_train,y_train)
    #ada_params=dict(gridsearch.best_params_)
    params=paramGrid
  elif model =="xgboost":
    paramGrid={'max_depth':[5,10],
            'min_child_weight':np.arange(1,9,1),
            'gamma':np.arange(0,1,0.001),
            'subsample':np.arange(0.1,0.9,0.05),
            'colsample_bytree':np.arange(0.1,0.9,0.05),
            'n_estimator':[50,100,200],
            'objective':['binary:logistic','binary:logitraw'],
            'learning_rate':[0.001,0.01,0.1]}
    model_run = XGBClassifier()
    #print(paramGrid)
    #gridsearch=GSCV(model_run,paramGrid,verbose=1,cv=KFold(n_splits=10,random_state=None).get_n_splits([X_train,y_train]))
    #gridsearch.fit(X_train,y_train)
    #xgb_params=dict(gridsearch.best_params_)
    #params=xgb_params
    params=paramGrid
    #output = run_the_model()
  return model_run


In [0]:
import csv

In [0]:
model = train_the_model('RandomForest')
model.fit(X_train, y_train)
m = model.predict(X_test)
m_new = m.transpose()

#print(m_new)
#print(y_train)
#clf.fit(X_train, y_train)
#score = clf.score(X_test, y_test)
score = model.score(X_test, y_test)
print(score)
#ax = plt.subplot(1, 2, 1)
#clf.fit(X_train, y_train)
#score = clf.score(X_test, y_test)
#results_new = pd.concat((results, m),join='outer',axis=1)
#print(results)
#print(m_new.size)
#temp=[]
import csv
#print(len(X_test))
i=0
data = []
for i in range(len(results)):
  temp = results.iloc[i] + '    ' + str(m_new[i])
  data.append(temp)

with open("submission.csv","w") as f:
    wr = csv.writer(f,delimiter="\n")
    for ele in data:
        wr.writerow([ele+","])


1.0
